In [1]:
!which python

/home/user/jfayzullaev/stellar-clustering/.venv-vis/bin/python


In [2]:
import os
import pandas as pd
import numpy as np

from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import normalized_mutual_info_score as NMI, adjusted_rand_score as ARI



In [3]:
SPECTRAL_FILE_TX = "transactions/spectral_labels_all_k.csv"
SPECTRAL_FILE_TL = "trustlines/spectral_labels_all_k.csv"

In [14]:
NORM_LABELS_PATH = "~/stellar-clustering/network/labled-data/labels/label-normalization/labels_entities_normalized.csv"  

In [ ]:
N_SPLITS = 5
RANDOM_STATE = 42

In [ ]:
def overall_purity_comm(df_comm_name: pd.DataFrame, comm_col: str, name_col: str = "name") -> float:

    if df_comm_name.empty:
        return np.nan
    counts = df_comm_name.groupby([comm_col, name_col]).size().reset_index(name='cnt')
    totals = counts.groupby(comm_col)['cnt'].sum()
    max_per_comm = counts.groupby(comm_col)['cnt'].max()
    return float(max_per_comm.sum() / totals.sum())

In [ ]:
def load_kmeans_fixed_k(spectral_file: str, k_col: str) -> pd.DataFrame:

    df = pd.read_csv(spectral_file)

    if k_col not in df.columns:

        candidates = [c for c in df.columns if c != 'account_id']
        raise ValueError(
            f"not found '{k_col}'. "
            f"found: {sorted(candidates)}"
        )

    df = df[['account_id', k_col]].dropna().drop_duplicates()
    df = df.rename(columns={k_col: 'cluster'})

    # Cast type
    try:
        df['account_id'] = df['account_id'].astype(int)
    except Exception:
        df['account_id'] = df['account_id'].astype(str)


    try:
        df['cluster'] = df['cluster'].astype(int)
    except Exception:
        tmp = pd.to_numeric(df['cluster'], errors='coerce')
        max_c = int(np.nanmax(tmp)) if np.isfinite(tmp).any() else -1
        tmp = tmp.fillna(max_c + 1).astype(int)
        df['cluster'] = tmp

    return df

In [ ]:
def evaluate_fixed_spectral_cv(
    labels_path: str,
    clusters_df: pd.DataFrame,
    label_col: str = "name",
    n_splits: int = 5,
    random_state: int = 42
):


    labels = (pd.read_csv(labels_path)
                .dropna(subset=['account_id', label_col])
                .drop_duplicates(subset=['account_id'])
                .rename(columns={label_col: 'name'}))

    try:
        labels['account_id'] = labels['account_id'].astype(int)
        clu = clusters_df.copy()
        clu['account_id'] = clu['account_id'].astype(int)
    except Exception:
        labels['account_id'] = labels['account_id'].astype(str)
        clu = clusters_df.copy()
        clu['account_id'] = clu['account_id'].astype(str)

    # join only accounts present in this Spectral
    joined = labels.merge(clu, on='account_id', how='inner')

    n_labeled = len(labels)
    n_joined  = len(joined)
    coverage  = (n_joined / n_labeled) if n_labeled else 0.0
    if n_joined == 0:
        raise ValueError("No labeles found sk after join")


    le = LabelEncoder()
    y_all = le.fit_transform(joined['name'].values)
    X_ids = joined['account_id'].values

    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=random_state)
    rows = []

    for fold, (tr_idx, te_idx) in enumerate(skf.split(X_ids, y_all), start=1):
        df_tr = joined.iloc[tr_idx].copy()
        df_te = joined.iloc[te_idx].copy()

        y_true_tr = le.transform(df_tr['name'])
        y_pred_tr = df_tr['cluster'].values
        nmi_tr = NMI(y_true_tr, y_pred_tr)
        ari_tr = ARI(y_true_tr, y_pred_tr)
        purity_tr = overall_purity_comm(df_tr[['cluster', 'name']], comm_col='cluster', name_col='name')

        y_true_te = le.transform(df_te['name'])
        y_pred_te = df_te['cluster'].values
        nmi_te = NMI(y_true_te, y_pred_te)
        ari_te = ARI(y_true_te, y_pred_te)
        purity_te = overall_purity_comm(df_te[['cluster', 'name']], comm_col='cluster', name_col='name')

        rows.append({
            'fold': fold,
            'n_train': len(df_tr),
            'n_test': len(df_te),
            'train_frac': len(df_tr) / len(joined),
            'NMI_train': nmi_tr,
            'ARI_train': ari_tr,
            'Purity_train': purity_tr,
            'NMI_test': nmi_te,
            'ARI_test': ari_te,
            'Purity_test': purity_te,
        })

    per_fold_df = pd.DataFrame(rows)

    averages = {
        'Avg_NMI_train': float(np.nanmean(per_fold_df['NMI_train'].values)),
        'Avg_ARI_train': float(np.nanmean(per_fold_df['ARI_train'].values)),
        'Avg_Purity_train': float(np.nanmean(per_fold_df['Purity_train'].values)),
        'Avg_NMI_test': float(np.nanmean(per_fold_df['NMI_test'].values)),
        'Avg_ARI_test': float(np.nanmean(per_fold_df['ARI_test'].values)),
        'Avg_Purity_test': float(np.nanmean(per_fold_df['Purity_test'].values)),
        'Avg_train_frac': float(np.nanmean(per_fold_df['train_frac'].values)),
    }

    coverage_info = {
        'n_labeled': int(n_labeled),
        'n_joined': int(n_joined),
        'coverage': float(coverage),
    }

    return per_fold_df, averages, coverage_info


## Transactions

In [11]:
K_COL = "cluster_k65" 

In [ ]:
clusters_df = load_kmeans_fixed_k(SPECTRAL_FILE_TX, K_COL)

In [15]:
norm_labels_path = os.path.expanduser(NORM_LABELS_PATH)

In [16]:
norm_per_fold, norm_avg, norm_cov = evaluate_fixed_spectral_cv(
    labels_path=norm_labels_path,
    clusters_df=clusters_df,
    label_col="name",
    n_splits=N_SPLITS,
    random_state=RANDOM_STATE
)

/home/user/jfayzullaev/stellar-clustering/.venv-vis/lib/python3.9/site-packages/sklearn/model_selection/_split.py:805: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


In [17]:
print(f"\n Spectral {K_COL} | Normalized Labels per fold metrics")
display(norm_per_fold)


 Spectral cluster_k65 | Normalized Labels per fold metrics


,fold,n_train,n_test,train_frac,NMI_train,ARI_train,Purity_train,NMI_test,ARI_test,Purity_test
0,1,317,80,0.798489,0.327094,0.055765,0.757098,0.381342,0.071749,0.762500
1,2,317,80,0.798489,0.351165,0.078969,0.763407,0.307745,-0.001839,0.737500
2,3,318,79,0.801008,0.341348,0.064882,0.761006,0.329755,0.039045,0.746835
3,4,318,79,0.801008,0.327944,0.052710,0.751572,0.386594,0.103329,0.784810
4,5,318,79,0.801008,0.325160,0.060389,0.757862,0.391701,0.085360,0.759494


In [18]:
print(f"\n Spectral {K_COL} | Normalized Labels avgs")
for k, v in norm_avg.items():
    print(f"{k}: {v:.6f}")
print(f"Coverage: {norm_cov['coverage']:.2%}  ({norm_cov['n_joined']}/{norm_cov['n_labeled']})")



 Spectral cluster_k65 | Normalized Labels avgs
Avg_NMI_train: 0.334542
Avg_ARI_train: 0.062543
Avg_Purity_train: 0.758189
Avg_NMI_test: 0.359427
Avg_ARI_test: 0.059529
Avg_Purity_test: 0.758228
Avg_train_frac: 0.800000
Coverage: 4.90%  (397/8102)


In [19]:
TX_PATH = 'transactions/cross-validation'
os.makedirs(TX_PATH, exist_ok=True)

In [20]:
norm_per_fold.to_csv(f"{TX_PATH}/tx_cv_{K_COL}_norm_per_fold.csv", index=False)
pd.DataFrame([{
    **norm_avg,
    **norm_cov,
    "k_col": K_COL,
    "source_file": os.path.basename(SPECTRAL_FILE_TX)
}]).to_csv(f"{TX_PATH}/tx_cv_{K_COL}_norm_summary.csv", index=False)

print(f"Saved to {TX_PATH}")



Saved to transactions/cross-validation


## Trustlines

In [22]:
K_COL = "cluster_k65" 

In [ ]:
clusters_df = load_kmeans_fixed_k(SPECTRAL_FILE_TL, K_COL)

In [24]:
norm_labels_path = os.path.expanduser(NORM_LABELS_PATH)

In [25]:
norm_per_fold, norm_avg, norm_cov = evaluate_fixed_spectral_cv(
    labels_path=norm_labels_path,
    clusters_df=clusters_df,
    label_col="name",
    n_splits=N_SPLITS,
    random_state=RANDOM_STATE
)

/home/user/jfayzullaev/stellar-clustering/.venv-vis/lib/python3.9/site-packages/sklearn/model_selection/_split.py:805: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


In [26]:
print(f"\n Spectral {K_COL} | Normalized Labels per fold metrics")
display(norm_per_fold)


 Spectral cluster_k65 | Normalized Labels per fold metrics


,fold,n_train,n_test,train_frac,NMI_train,ARI_train,Purity_train,NMI_test,ARI_test,Purity_test
0,1,111,28,0.798561,0.285551,0.305131,0.891892,0.000000,0.000000,0.857143
1,2,111,28,0.798561,0.200071,0.211736,0.882883,0.404275,0.344569,0.892857
2,3,111,28,0.798561,0.256127,0.213749,0.891892,0.404275,0.344569,0.892857
3,4,111,28,0.798561,0.233633,0.212987,0.891892,0.404275,0.344569,0.892857
4,5,112,27,0.805755,0.259239,0.265468,0.875000,0.000000,0.000000,0.925926


In [27]:
print(f"\n Spectral {K_COL} | Normalized Labels avgs")
for k, v in norm_avg.items():
    print(f"{k}: {v:.6f}")
print(f"Coverage: {norm_cov['coverage']:.2%}  ({norm_cov['n_joined']}/{norm_cov['n_labeled']})")



 Spectral cluster_k65 | Normalized Labels avgs
Avg_NMI_train: 0.246924
Avg_ARI_train: 0.241814
Avg_Purity_train: 0.886712
Avg_NMI_test: 0.242565
Avg_ARI_test: 0.206742
Avg_Purity_test: 0.892328
Avg_train_frac: 0.800000
Coverage: 1.72%  (139/8102)


In [29]:
TL_PATH = 'trustlines/cross-validation'
os.makedirs(TL_PATH, exist_ok=True)

In [30]:
norm_per_fold.to_csv(f"{TL_PATH}/tx_cv_{K_COL}_norm_per_fold.csv", index=False)
pd.DataFrame([{
    **norm_avg,
    **norm_cov,
    "k_col": K_COL,
    "source_file": os.path.basename(SPECTRAL_FILE_TL)
}]).to_csv(f"{TL_PATH}/tx_cv_{K_COL}_norm_summary.csv", index=False)

print(f"Saved to {TL_PATH}")

Saved to trustlines/cross-validation
